In [1]:
from pycaret.datasets import get_data
from pycaret.regression import *

import pandas as pd
import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib as mpl


mpl.rcParams["figure.dpi"] = 300
mpl.rcParams["xtick.labelsize"] = 6
mpl.rcParams["ytick.labelsize"] = 6
mpl.rcParams["axes.labelsize"] = 6
mpl.rcParams["legend.fontsize"] = 6
mpl.rcParams["legend.title_fontsize"] = 6
# mpl.rcParams["legend.shadow"] = True


# for key, value in mpl.rcParams.items():
#     print(key, "=", value)


> Data

This is a health insurance dataset, where the features are various attributes including age, sex, Body Mass Index (BMI), whether the person is a smoker or not, number of children and US region. Furthermore, the dataset target variable is the billed charges for every individual


In [2]:
data = get_data("insurance")
# data.select_dtypes(include=["number"]).hist(bins=30, figsize=(6, 4))
# plt.show()

numeric = ["age", "bmi", "children", "charges"]
# data[numeric].hist(bins=20, figsize = (12,8))
# plt.show()


In [29]:
categorical = ["smoker", "sex", "region"]
color = ["C0", "C1", "C2", "C3"]
# fig, axes = plt.subplots(2, 2, figsize=(12, 10))
# axes[1, 1].set_axis_off()
# for ax, col in zip(axes.flatten(), categorical):
#     data[col].value_counts().plot(kind="bar", ax=ax, color=color)
#     ax.set_xlabel(col)


> Regression A

Relationship between numeric and categorical variables by using hue mapping. In this case, we plot the target variable histogram, colored differently for every category of the smoker, sex and region variables. Evidently, smokers get significantly higher charges compared to non-smokers. This is expected, as the health risks associated with smoking are
numerous and well-documented.


In [4]:
# fig, axes = plt.subplots(2, 2, figsize=(9, 6))
# axes[1, 1].set_axis_off()
# for ax, col in zip(axes.flatten(), categorical):
#     sns.histplot(data, x="charges", hue=col, multiple="stack", ax=ax)


> Regression B

Scatter plots are a type of visualization that helps us understand the relationship between numeric variables. The pairplot() Seaborn function creates a grid of scatter plots for all pairs of numeric variables in a given dataset. The diagonal contains distribution plots of the variables, such as histograms or KDE plots in this case. Once again, we use hue mapping to highlight the differences between smokers and nonsmokers. As we can see, age is correlated with charges, i.e. people get higher charges as they grow older. In spite of that, being a non-smoker keeps the cost lower for most people, regardless of their age. Furthermore, overweight and obese people don’t seem
to get significantly higher charges, unless they are also smokers.


In [5]:
cols = ["age", "bmi", "charges", "smoker"]
# sns.pairplot(data[cols], hue='smoker')
# plt.show()


> Pycaret environment


In [49]:
reg = setup(
    data=data,
    target="charges",
    train_size=0.8,
    session_id=123,
    numeric_features=numeric[:-1],
    categorical_features=categorical,
    transformation=True,
    normalize=True,
    transform_target=True,
    # one_hot_encoding=True
)


In [43]:
# model = create_model('gbr', fold = 10)


In [41]:
get_config("X").head()


,age,sex,bmi,children,smoker,region
67,40,male,26.315001,1,no,northwest
736,37,female,38.389999,0,yes,southeast
310,50,male,26.600000,0,no,southwest
963,46,male,24.795000,3,no,northeast
680,21,female,17.400000,1,no,southwest


In [34]:
# get_config("X_train").head()
# get_config('X_transformed')
# get_config()


In [ ]:
best = compare_models(sort="RMSE")
